In [ ]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 8, 3, padding=1) #input 30*30*1 OUtput? 28*28*8
        self.conv2 = nn.Conv2d(8, 8, 3, padding=1) #input 30*30*8 OUtput? 28*28*8
        self.pool1 = nn.MaxPool2d(2, 2) #input 28*28*8 OUtput? 14*14*8
        self.bn1   = nn.BatchNorm2d(8)
        self.conv3 = nn.Conv2d(8, 16, 3, padding=1) #input 16*16*8 OUtput? 14*14*16
        self.conv4 = nn.Conv2d(16, 16, 3, padding=1) #input 16*16*16 OUtput? 14*14*16
        self.pool2 = nn.MaxPool2d(2, 2) #input 14*14*16 OUtput?7*7*16
        self.bn2   = nn.BatchNorm2d(16)
        self.conv5 = nn.Conv2d(16, 32, 3) #input7*7*16 OUtput?5*5*32
        self.conv6 = nn.Conv2d(32, 10, 1)
        self.gap = nn.AvgPool2d(kernel_size = 5)#input5*5*32 OUtput?1*1*32
        
        


    def forward(self, x):
        x = self.bn1(self.pool1(F.relu(self.conv2(F.relu(self.conv1(x))))))
        x = self.bn2(self.pool2(F.relu(self.conv4(F.relu(self.conv3(x))))))
        x = F.relu(self.conv5(x))
        x = self.conv6(x)
        x = self.gap(x)
        x = x.view(-1,10)
        return F.log_softmax(x)
        

NameError: ignored

In [ ]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

NameError: ignored

In [ ]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw


Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw


Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw


Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw
Processing...
Done!


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:469: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [ ]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [ ]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 20):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.1002, Accuracy: 9728/10000 (97.28%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0780, Accuracy: 9751/10000 (97.51%)



  0%|          | 0/469 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0363, Accuracy: 9892/10000 (98.92%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0341, Accuracy: 9892/10000 (98.92%)



loss=0.05447503551840782 batch_id=468: 100%|██████████| 469/469 [00:12<00:00, 36.33it/s]



Test set: Average loss: 0.0455, Accuracy: 9856/10000 (98.56%)

